In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession .\
builder .\
appName("spark") .\
config("spark.sql.warehouse.dir","/user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [3]:
spark

In [4]:
spark.sql("use data_analyst")

""


In [6]:
spark.sql("show tables")

database,tableName,isTemporary
data_analyst,customers,false
data_analyst,loans_delinq,false
data_analyst,loans_enq,false
data_analyst,loans_repayments,false


In [11]:
spark.sql("select * from customers limit 3")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
45c47c49ef9346a50...,area sales manager,5,MORTGAGE,48000.0,NY,131xx,USA,C,C2,Source Verified,221587.0,Joint App,90000.0,Source Verified,2023-09-15 17:18:...
ae8702f966762bdde...,Manager,7,MORTGAGE,105000.0,NJ,087xx,USA,C,C3,Source Verified,51300.0,Individual,null,null,2023-09-15 17:18:...
bfdd1a23cef47a143...,Admitting,9,MORTGAGE,70000.0,OR,970xx,USA,B,B2,Source Verified,331700.0,Individual,null,null,2023-09-15 17:18:...


In [10]:
spark.sql("select * from loans_delinq limit 3")

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,0
aac68850fdac09fd0...,1,0.0,0
c89986155a070db2e...,1,0.0,15


In [12]:
spark.sql("select * from loans_enq limit 3")

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
673da0da820de70f9...,0,0,1
f5725a82fea842f9b...,0,0,2
0e697722917e34328...,0,0,2


In [13]:
spark.sql("select * from loans_repayments limit 3")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
28674390,7884.28,5178.41,0.0,14329.62,482.23,Nov-2016,null,2023-09-19 05:25:...
28563513,9000.0,955.61,0.0,9955.61,6420.29,Nov-2015,null,2023-09-19 05:25:...
28694706,22125.0,751.39,0.0,22876.39,22073.85,Dec-2014,null,2023-09-19 05:25:...


In [22]:
spark.sql("""create Or Replace view final as 
select
    c.member_id,
    c.emp_title,
    c.emp_length,
    c.home_ownership,
    c.annual_income,
    c.address_state,
    c.address_zipcode,
    c.address_country,
    c.grade,
    c.sub_grade,
    c.verification_status,
    c.total_high_credit_limit,
    c.application_type,
    c.join_annual_income,
    c.verification_status_joint,
    c.ingest_date AS customer_ingest_date,
    l1.member_id as deli,
    l2.member_id as loan_del,
    l1.delinq_2yrs,
    l1.delinq_amnt,
    l1.mths_since_last_delinq,
    l2.pub_rec,
    l2.pub_rec_bankruptcies,
    l2.inq_last_6mths
    
    from customers as c left join loans_delinq as l1 on c.member_id = l1.member_id 
    left join loans_enq as l2 on l1.member_id = l2.member_id 

""")

""


In [24]:
spark.sql("select * from final")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,customer_ingest_date,deli,loan_del,delinq_2yrs,delinq_amnt,mths_since_last_delinq,pub_rec,pub_rec_bankruptcies,inq_last_6mths
000c8875b71a6b47c...,Director of Gvt O...,1,MORTGAGE,100000.0,CA,920xx,USA,A,A4,Not Verified,761500.0,Individual,null,null,2023-09-15 17:18:...,000c8875b71a6b47c...,000c8875b71a6b47c...,1,0.0,0,0,0,0
003769d7f54c7859e...,Trainer,10,MORTGAGE,35000.0,NC,282xx,USA,E,E1,Verified,184961.0,Individual,null,null,2023-09-15 17:18:...,003769d7f54c7859e...,003769d7f54c7859e...,1,0.0,0,0,0,0
003e1e6cbd2920bbb...,Operations Superv...,9,MORTGAGE,92000.0,LA,705xx,USA,A,A5,Source Verified,375806.0,Individual,null,null,2023-09-15 17:18:...,003e1e6cbd2920bbb...,003e1e6cbd2920bbb...,1,0.0,0,1,1,0
004017b21bd4d6271...,Regional Rehab Di...,9,MORTGAGE,113000.0,NC,276xx,USA,C,C1,Not Verified,120587.0,Individual,null,null,2023-09-15 17:18:...,004017b21bd4d6271...,004017b21bd4d6271...,6,0.0,15,0,0,0
005b4c3db3fce07dc...,office manager,9,MORTGAGE,61000.0,FL,334xx,USA,B,B5,Source Verified,28386.0,Individual,null,null,2023-09-15 17:18:...,005b4c3db3fce07dc...,005b4c3db3fce07dc...,1,0.0,0,1,0,1
00710707c563c2119...,owner,3,MORTGAGE,109000.0,FL,324xx,USA,B,B5,Source Verified,356897.0,Individual,null,null,2023-09-15 17:18:...,00710707c563c2119...,00710707c563c2119...,1,0.0,0,0,0,0
007da79904f69970d...,Senior Agile Coach,1,RENT,195000.0,CA,950xx,USA,A,A1,Verified,93098.0,Individual,null,null,2023-09-15 17:18:...,007da79904f69970d...,007da79904f69970d...,1,0.0,0,0,0,0
00f435a80d0440ece...,Security,1,MORTGAGE,200000.0,NY,109xx,USA,C,C2,Source Verified,441644.0,Individual,null,null,2023-09-15 17:18:...,00f435a80d0440ece...,00f435a80d0440ece...,3,0.0,0,0,0,0
00fc2ae3ffb1213e4...,Service Tech.,10,MORTGAGE,44000.0,IL,600xx,USA,C,C3,Source Verified,165788.0,Individual,null,null,2023-09-15 17:18:...,00fc2ae3ffb1213e4...,00fc2ae3ffb1213e4...,2,0.0,0,0,0,0
00fc8144cb210ba8c...,Operations manager,3,MORTGAGE,64000.0,VA,233xx,USA,C,C1,Source Verified,265491.0,Individual,null,null,2023-09-15 17:18:...,00fc8144cb210ba8c...,00fc8144cb210ba8c...,1,0.0,0,1,1,0


In [23]:
spark.sql("show tables")

database,tableName,isTemporary
data_analyst,customers,false
data_analyst,final,false
data_analyst,loans_delinq,false
data_analyst,loans_enq,false
data_analyst,loans_repayments,false


In [25]:
spark.sql("""create table final1 as 
select
    c.member_id,
    c.emp_title,
    c.emp_length,
    c.home_ownership,
    c.annual_income,
    c.address_state,
    c.address_zipcode,
    c.address_country,
    c.grade,
    c.sub_grade,
    c.verification_status,
    c.total_high_credit_limit,
    c.application_type,
    c.join_annual_income,
    c.verification_status_joint,
    c.ingest_date AS customer_ingest_date,
    l1.member_id as deli,
    l2.member_id as loan_del,
    l1.delinq_2yrs,
    l1.delinq_amnt,
    l1.mths_since_last_delinq,
    l2.pub_rec,
    l2.pub_rec_bankruptcies,
    l2.inq_last_6mths
    
    from customers as c left join loans_delinq as l1 on c.member_id = l1.member_id 
    left join loans_enq as l2 on l1.member_id = l2.member_id 

""")

""


In [ ]:
spark.sql("selec")